In [ ]:
import pandas
import itertools as it
import string
from collections import Counter
import json
import re
import numpy
from sklearn.externals import joblib

In [ ]:
with open('data/stopwords.txt') as fh:
    stopwords = set(fh.read().splitlines())

In [ ]:
clf = joblib.load('data/twitter_classifier_best.pkl')

In [ ]:
embeddings = {}
with open('data/glove.twitter.27B.25d.txt') as fh:
    for line in fh:
        line = line.strip().split(" ")
        embeddings[line[0]] = [float(t) for t in line[1:]]

In [ ]:
def normalise(text):
    
    text = text.lower()
    text = text.replace("\n", " ")
    text = re.sub(r"https?:\/\/\S+\b|www\.(\w+\.)+\S*", " <url> ", text)
    text = text.replace("/", " / ")
    text = re.sub(r"@\w+", " <user> ", text)
    text = re.sub(r"[8:=;]['`\-][)d]+|[)d]+['`\-][8:=;]", " <smile> ", text)
    text = re.sub(r"[8:=;]['`\-]p+", " <lolface> ", text)
    text = re.sub(r"[8:=;]['`\-]\(+|\)+['`\-][8:=;]", " <sadface> ", text)
    text = re.sub(r"[8:=;]['`\-][\/|l*]", " <neutralface> ", text)
    text = text.replace(r"<3", " <heart> ")
    text = re.sub(r"[-+]?[.\d]*[\d]+[:,.\d]*", " <number> ", text)
    text = text.replace(r"#", " <hashtag> ")
    text = re.sub(r"([!?.,()])+", r" \1 ", text)
    
    return text

In [ ]:
def tokenise(text):
    return tuple([t.strip() for t in text.strip().split()])

In [ ]:
def parse_treatment_definitons(definion_file):
    treatment_set = set()
    treatment_mapping = {}
    max_length = 1
    for line in definion_file:
        line = line.strip()
        treatments = line.split(',')
        name = treatments[0]
        for treatment in treatments:
            treatment = tuple(tokenise(normalise(treatment)))
            max_length = max(len(treatment), max_length)
            treatment_set.add(treatment)
            treatment_mapping[treatment] = name
    return treatment_set, treatment_mapping, max_length

In [ ]:
def window_sliding(iterable, n):
    gens = (
        it.chain(it.repeat(None, n - 1 - i), iterable, it.repeat(None, i))
        for i, gen in enumerate(it.tee(iterable, n)))
    return zip(*gens)

def find_treatments(tokens):
    found_treatments = []
    for x in range(max_treatment_length, 0, -1):
        for window in window_sliding(tokens, x):
            if tuple(window) in treatment_set:
                found_treatments.append(treatment_mapping[tuple(window)])
    return list(set(found_treatments)) or None

In [ ]:
def filter_stopwords(tokens):
    return [t for t in tokens if len(t) > 2 and t not in stopwords]

In [ ]:
def calculate_embeddings(tokens):
    token_embdgs = [embeddings.get(t) for t in tokens]
    token_embdgs = [t for t in token_embdgs if t is not None]
    return numpy.mean(token_embdgs, axis=0)

In [ ]:
with open('tweets.jsonl', 'r') as fh:
    df = pandas.read_json(fh.read(), lines=True, convert_dates=True)

In [ ]:
df = df.loc[(df['retweeted_status'].isnull()) & (df['lang'] == "en")]

In [ ]:
df = df.loc[(df.created_at >= "2017-12-04") & (df.created_at <= "2018-02-03")]

In [ ]:
df["created_at"].groupby(df.created_at.dt.weekofyear).count()

In [ ]:
df["extended_tweet"] = df["extended_tweet"].apply(lambda x: x["full_text"]  if type(x) == dict else x )

In [ ]:
df["text"] = df["extended_tweet"].combine_first(df["text"])

In [ ]:
df["text"] = df["text"].apply(lambda x: x.replace("\n", " "))

In [ ]:
df["user_name"] = df["user"].apply(lambda x: x["screen_name"])

In [ ]:
df = df.loc[:, ["id", "created_at", "text", "retweet_count", "user_name"]]

In [ ]:
df["tokens"] = df["text"].apply(lambda x: tokenise(normalise(x)))

In [ ]:
len(df.user_name.unique())

In [ ]:
treatment_set, treatment_mapping, max_treatment_length = parse_treatment_definitons(open("data/treatment_definitons.txt", 'r'))

In [ ]:
df['filtered_tokens'] = df['tokens'].apply(filter_stopwords)

In [ ]:
word_counts = Counter()
for elem in df["filtered_tokens"]:
    word_counts += Counter(elem)

In [ ]:
word_counts.most_common(100)

In [ ]:
df['treatments'] = df['tokens'].apply(find_treatments)

In [ ]:
df = df.loc[df['treatments'].notnull()]

In [ ]:
df['prediction'] = df['text'].apply(lambda t: clf.predict([calculate_embeddings(t)])[0])

In [ ]:
df['prediction'].groupby(df.prediction).count()

In [ ]:
unique_text_counts = df["tokens"].groupby(df.tokens).count().sort_values(ascending=False).values

In [ ]:
unique_text_counts.mean()

In [ ]:
numpy.median(unique_text_counts)

In [ ]:
len(df.user_name.unique())

In [ ]:
df.to_csv("tweets_with_treatments.csv")

In [ ]:
treatment_counts = Counter()
for elem in df["treatments"]:
    treatment_counts += Counter(elem)

In [ ]:
treatment_counts.most_common(100)